### Prompt

In [1]:
import bs4, tiktoken, numpy as np, os

from langchain import hub
from langchain.load import dumps, loads
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

learn_api_key = os.environ['LANGCHAIN_API_KEY']
openai_api_key = os.environ['OPENAI_API_KEY']
groq_api_key = os.environ['GROQ_API_KEY']

In [2]:
### INDEXING ###
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=50)

# Make Splits
splits = text_splitter.split_documents(blog_docs)

# Index
vectorstore = Chroma.from_documents(
    splits,
    HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
)
retriever = vectorstore.as_retriever()

In [3]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel's was born in what country?",
        "output": "what is Jan Sindel's personal history?",
    },
]

# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [4]:
generate_queries_step_back = prompt | ChatGroq(model="gemma2-9b-it", api_key=groq_api_key, temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question":question})

"How do large language model agents break down complex tasks into smaller, manageable steps? \n\n\nLet me know if you'd like me to elaborate on any of these concepts! \n\n"

In [5]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
        # Retrieve context using step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGroq(model="gemma2-9b-it", api_key=groq_api_key, temperature=0)
    | StrOutputParser()
)

chain.invoke({"question":question})

'Task decomposition is a crucial component of planning for LLM-powered autonomous agents. \n\nIt involves breaking down large, complex tasks into smaller, more manageable subgoals. This makes it easier for the agent to:\n\n* **Handle complexity:**  Large tasks can be overwhelming for LLMs. Decomposition allows them to focus on one step at a time, improving their ability to solve complex problems.\n* **Improve efficiency:** By tackling smaller tasks individually, the agent can allocate resources more effectively and avoid getting bogged down in unnecessary steps.\n* **Enable reasoning:**  Decomposition often reveals the underlying structure of a problem, allowing the LLM to apply reasoning and logic to each subgoal.\n\nThere are several techniques for task decomposition in LLM agents:\n\n* **Chain of Thought (CoT):**  The LLM is prompted to "think step-by-step," breaking down the task into a sequence of logical steps.\n* **Tree of Thoughts (ToT):**  This method extends CoT by exploring 